In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
import scipy.io as sp
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score

dataset = sp.loadmat('breastw.mat')

X = dataset['X']
y = dataset['y']
y=np.reshape(y,(683,))

K=np.array([0.5,0.514,0.543,0.571,0.6,0.629,0.12,0.25])
Distance=np.array(['cityblock','manhattan','l1','euclidean','manhattan','euclidean','l2','l2'])
Contamination=np.array([0.357,0.357,0.357,0.357,0.357,0.347,0.22,0.43])

skf= StratifiedKFold(n_splits=12,shuffle=False,random_state=None)


for i in range(0,8) :  
    test=False
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        LOF = LocalOutlierFactor(n_neighbors=round(len(X_test)*K[i]), algorithm='auto', contamination=Contamination[i], metric=Distance[i])
        y_pred = LOF.fit_predict(X_test)
        X_scores = -LOF.negative_outlier_factor_
        y_pred[y_pred>0]=0
        y_pred[y_pred<0]=1
        F1=round(f1_score(y_test,y_pred),3)
        if test == False :
            matrice = np.array([[F1]])
            test = True
        else :
            matrice = np.append(matrice,np.array([F1]).reshape(1,1),axis=0)
    if i == 0  :
        df=pd.DataFrame(matrice,columns=['[{},{},{}]'.format(K[i],Distance[i],round(Contamination[i],3))])
    else :
        df1=pd.DataFrame(matrice,columns=['[{},{},{}]'.format(K[i],Distance[i],round(Contamination[i],3))])
        df=pd.concat([df, df1], axis = 1)
display(df)